## Import dependencies

In [1]:
import sys
import os
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import numpy as np
from gensim.models import KeyedVectors
import mlflow
from pycaret.classification import predict_model
from IPython.core.display import display, Markdown

from sem_covid.services.sc_wrangling.evaluation_metrics import model_evaluation_metrics

from sem_covid.services.model_registry import ClassificationModel, get_best_model_from_ml_flow
from sem_covid.services.store_registry import StoreRegistry
from sem_covid.services.data_registry import Dataset, LanguageModel
from sem_covid.services.sc_wrangling.mean_vectorizer import text_to_vector
import pandas as pd
import regex as re

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/conda/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
/opt/conda/

## Define consts

In [2]:
EU_CELLAR_TEXT_COLUMNS = ['title', 'content']

EU_CELLAR_NEW_COLUMNS = ['businesses', 'citizens', 'workers']

In [59]:
runs_df = mlflow.search_runs(experiment_ids=['12'], filter_string="tags.Source='finalize_model'")

In [60]:
runs_df.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.TT,metrics.F1,metrics.Recall,metrics.AUC,metrics.Kappa,metrics.MCC,metrics.Prec.,metrics.Accuracy,params.max_features,params.min_samples_split,params.max_leaf_nodes,params.max_samples,params.bootstrap,params.random_state,params.verbose,params.ccp_alpha,params.max_depth,params.min_impurity_decrease,params.min_weight_fraction_leaf,params.warm_start,params.min_impurity_split,params.n_jobs,params.class_weight,params.criterion,params.n_estimators,params.oob_score,params.min_samples_leaf,tags.Run Time,tags.USI,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.type,tags.URI,tags.mlflow.log-model.history,tags.Run ID,tags.Source,tags.mlflow.user
0,fcc1b197848e44b2ba895bd9725f7432,12,FINISHED,s3://mlflow/12/fcc1b197848e44b2ba895bd9725f743...,2021-06-12 08:14:20.809000+00:00,2021-06-12 08:14:21.280000+00:00,0.05,0.5911,0.5731,0.7632,0.3777,0.3798,0.616,0.7184,sqrt,9,None,None,False,6610,0,0.0,9,0.0002,0.0,False,None,-1,balanced_subsample,entropy,10,False,6,0.36,6039,Random Forest Classifier,/opt/conda/lib/python3.8/site-packages/ipykern...,LOCAL,731b92ea,"[{""run_id"": ""fcc1b197848e44b2ba895bd9725f7432""...",fcc1b197848e44b2ba895bd9725f7432,finalize_model,jovyan


In [24]:
feature_store = StoreRegistry.es_feature_store()
dataset_x = feature_store.get_features('fs_pwdb_tg1_x')
dataset_y = feature_store.get_features('fs_pwdb_tg1_y')

100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [7]:
model = get_best_model_from_ml_flow(experiment_ids=['12'])
#model = ClassificationModel.pwdb_by_class_name('businesses')

In [25]:
dataset = dataset_x
#dataset['businesses'] = dataset_y['businesses'].values

In [26]:
dataset['businesses']

KeyError: 'businesses'

In [27]:
predicted_dataset = predict_model(model, data=dataset)

In [29]:
predicted_dataset[['Label']].head(5)

,Label
_id,
0,0
1,0
2,1
3,0
4,1


In [30]:
metrics = model_evaluation_metrics(dataset_y['businesses'], predicted_dataset['Label'].values)
display(Markdown('businesses'))
display(pd.DataFrame(metrics, index=[0]))

businesses

,Accuracy,Precision,Recall,F1-Score,Mean Absolute Error,Mean Squared Error
0,1.0,1.0,1.0,1.0,0.0,0.0


## Load dataset and language model

In [3]:
df = Dataset.EU_CELLAR.fetch()


100% (9792 of 9792) |####################| Elapsed Time: 0:00:29 Time:  0:00:29


In [13]:
for i in range(0,10):
    print(df['content'][i][0:200])
























































Healthy Workplaces - Lighten the Load


Healthy Workplaces

LIGHTEN THE LOAD

Safety and health at work is everyone’s concern. It’s good for you.






































 

 

Ethics  
of Genome  
Editing 
#EthicsGroup_EU 

European Group  

on Ethics in Science and  

New Technologies 

 

 

 

Executive Summary of the Opinion 











































EUR XXXX EN

Environment
European Research on

Projects Funded by Horizon 2020 
(2014-2020)

and Health

Research and 
Innovation 



European Research on Env










C_2020350EN.01000101.xml


   	
               20.10.2020   

            	
               EN

            	
               Official Journal of the European Union

            	
            










C_2021053EN.01004601.xml


   	
               15.2.2021   

            	
               EN

            	
               Official Journal of the European Union

            	
         

In [17]:
df['content'] = df['content'].str.replace('\n','')

In [20]:
df['content'] = df['content'].apply(lambda x: re.sub(' +',' ',str(x)))

_id
0007dea694fcf194ad9af6ec8b597a5185330549e826f04c6ca3e91f8bad383b         Healthy Workplaces - Lighten the LoadHealthy W...
0013e1224a6385d0315875724287f0a044ec2050a8e710c14a479fa037eb9010          Ethics of Genome Editing #EthicsGroup_EU Euro...
00178f1c4c60992043347f38986f824c78096958d5bff593b8a1c79d058641f0         EUR XXXX ENEnvironmentEuropean Research onProj...
002086bd15c9aba8b8b3cdf88498e25735cf66043ddab8877aca317256615aff         C_2020350EN.01000101.xml \t 20.10.2020    \t E...
002b8d0e8dcdbbb3ad87c517e97e1632ced062ad5aeb0c0d8727a9791d560e65         C_2021053EN.01004601.xml \t 15.2.2021    \t EN...
                                                                                               ...                        
1a8f4a40f75f4c3bfabf8f15d04f93abb14caca67bd291979ed740af1cd178a8.json    CEF_Telecom_2018-3_flyer_web2018-3 CEF TELECOM...
1a9d9d70d292edb954293393c59fb34c2d611b6c3ffec5c8fc38fb051381c0e0.json    Research Project Report13 March 20202020/04Leb...
1ab088d32a8c

In [15]:
df.head(5)

,work,title,cdm_types,cdm_type_labels,resource_types,resource_type_labels,eurovoc_concepts,eurovoc_concept_labels,subject_matters,subject_matter_labels,directory_codes,directory_codes_labels,celex_numbers,legal_elis,id_documents,same_as_uris,authors,author_labels,full_ojs,oj_sectors,internal_comments,is_in_force,dates_document,dates_created,legal_dates_entry_into_force,legal_dates_signature,manifs_pdf,manifs_html,pdfs_to_download,htmls_to_download,content_path,content,failure_reason,language
_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0007dea694fcf194ad9af6ec8b597a5185330549e826f04c6ca3e91f8bad383b,http://publications.europa.eu/resource/cellar/...,Lighten the load : healthy workplaces.,[http://publications.europa.eu/ontology/cdm#pu...,[],[http://publications.europa.eu/resource/author...,[General publications],"[http://eurovoc.europa.eu/1280, http://eurovoc...","[occupational health, public awareness campaign]",[],[],[],[],[],[],[2019.3015],[http://publications.europa.eu/resource/genpub...,[],[],[],[],[],[],[2020-05-15],[],[],[],[http://publications.europa.eu/resource/cellar...,[],[http://publications.europa.eu/resource/cellar...,[/zip],[de9b6712c2491a089544c3429fcc46e81bbece4650b0c...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN
0013e1224a6385d0315875724287f0a044ec2050a8e710c14a479fa037eb9010,http://publications.europa.eu/resource/cellar/...,Executive summary of the opinion ethics of gen...,[http://publications.europa.eu/ontology/cdm#pu...,[],[http://publications.europa.eu/resource/author...,[General publications],[http://eurovoc.europa.eu/192],[health control],[],[],[],[],[],[],[2021.2075],[http://publications.europa.eu/resource/genpub...,[],[],[],[],[],[],[2021-04-20],[],[],[],[http://publications.europa.eu/resource/cellar...,[],[http://publications.europa.eu/resource/cellar...,[/zip],[549755ac598bdccfb0011ead80d9f1d8bc1a7949aca25...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN
00178f1c4c60992043347f38986f824c78096958d5bff593b8a1c79d058641f0,http://publications.europa.eu/resource/cellar/...,European research on environment and health : ...,[http://publications.europa.eu/ontology/cdm#pu...,[],[http://publications.europa.eu/resource/author...,"[General publications, Study]","[http://eurovoc.europa.eu/1005, http://eurovoc...","[EU financing, health control, health risk, me...",[],[],[],[],[],[],[2021.1657],[http://publications.europa.eu/resource/genpub...,[],[],[],[],[],[],[2021-04-09],[],[],[],[http://publications.europa.eu/resource/cellar...,[],[http://publications.europa.eu/resource/cellar...,[/zip],[f6b58991d75ce46cde7daa1563d11138212943aa1a25d...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN
002086bd15c9aba8b8b3cdf88498e25735cf66043ddab8877aca317256615aff,http://publications.europa.eu/resource/cellar/...,Opinion No 6/2020 (pursuant to Article 287(4) ...,[http://publications.europa.eu/ontology/cdm#le...,[],[http://publications.europa.eu/resource/author...,[Opinion],"[http://eurovoc.europa.eu/837, http://eurovoc....","[coronavirus disease, epidemic]",[http://publications.europa.eu/resource/author...,[Economic policy],[http://publications.europa.eu/resource/author...,[Instruments of economic policy],[52020AA0006],[],"[celex:52020AA0006, oj:JOC_2020_350_R_0001]",[http://publications.europa.eu/resource/celex/...,[http://publications.europa.eu/resource/author...,[European Court of Auditors],[http://publications.europa.eu/resource/cellar...,[5],[COVID19],[],[2020-09-07],[],[],[],[http://publications.europa.eu/resource/cellar...,[http://publications.europa.eu/resource/cellar...,[http://publications.europa.eu/resource/cellar...,[http://publications.europa.eu/resource/cellar...,[3fdbbae8dcc57b8744e15539ff19c1b1a005fa0240fd7...,\n\n\n\n\n\n\n\n\n\nC_2020350EN.01000101.xml\n...,NaN,NaN
002b8d0e8dcdbbb3ad87c517e97e1632ced062ad5aeb0c0d8727a9791d560e65,http://publications.europa.eu/resource/cellar/...,Case T-718/20: Action brought on 5 December 20...,[http://publications.europa.eu/ontology/cdm#re...,[],[ht

In [2]:
law2vec = LanguageModel.LAW2VEC.fetch()
law2vec_path = LanguageModel.LAW2VEC.path_to_local_cache()
l2v_dict = KeyedVectors.load_word2vec_format(law2vec_path, encoding="utf-8")

In [4]:
l2v_dict.vectors.shape[1]

200

## Prepare dataset

In [34]:
df['content'] = df['content'].apply(lambda x: ' '.join(x))

In [38]:
document = df['content'][2]

In [39]:
len(document)

687279

In [47]:
from sem_covid.services.sc_wrangling.data_cleaning import clean_fix_unicode, clean_to_ascii, clean_to_lower,
    clean_remove_stopwords, clean_remove_line_breaks, clean_remove_urls, clean_remove_emails

In [48]:
document = clean_fix_unicode(document)
document = clean_to_ascii(document)
document = clean_to_lower(document)
document = clean_remove_line_breaks(document)
document = clean_remove_urls(document, replace_with='')
document = clean_remove_emails(document, replace_with='')
document = clean_remove_stopwords(document)


In [51]:
document.encode(encoding='utf-8')

b'e u r x x x x e n e n v r o n m e n t e u r o p e n r e s e r c h o n p r o j e c t s f u n d e d b y h o r z o n 2 0 2 0 ( 2 0 1 4 - 2 0 2 0 ) n d h e l t h r e s e r c h n d n n o v t o n e u r o p e n r e s e r c h o n e n v r o n m e n t n d h e l t h : p r o j e c t s f u n d e d b y h o r z o n 2 0 2 0 ( 2 0 1 4 - 2 0 2 0 ) e u r o p e n c o m m s s o n d r e c t o r t e - g e n e r l f o r r e s e r c h n d n n o v t o n d r e c t o r t e d p e o p l e u n t d . 2 h e l t h n n o v t o n s & e c o s y s t e m s c o n t c t s o f e n r g e r e m l r t d - e - h - t e m @ e c . e u r o p . e u s o f e . n o r g e r @ e c . e u r o p . e u r t d - p u b l c t o n s @ e c . e u r o p . e u e u r o p e n c o m m s s o n b - 1 0 4 9 b r u s s e l s m n u s c r p t c o m p l e t e d n p r l 2 0 2 1 . 1 s t e d t o n . t h s d o c u m e n t h s b e e n p r e p r e d f o r t h e e u r o p e n c o m m s s o n , h o w e v e r t r e f l e c t s t h e v e w s o n l y o f t h e u t h o r s 

In [52]:
eu_timeline = Dataset.EU_ACTION_TIMELINE.fetch()

100% (171 of 171) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [59]:
eu_timeline

"\n\n\n\n\n\n\n\nToday, the Commission, in cooperation with the President of the European Council, has put forward a European roadmap to phase-out the containment measures due to the coronavirus outbreak.\n\nWhile we are still in firefighting mode, the necessary extraordinary measures taken by Member States and the EU are working. They have slowed down the spread of the virus and saved thousands of lives. However, these measures and the corresponding uncertainty come at a dramatic cost to people, society and the economy, and cannot last indefinitely.\n\nPresident of the European Commission Ursula von der Leyen said: “Saving lives and protecting Europeans from the coronavirus is our number one priority. At the same time, it is time to look ahead and to focus on protecting livelihoods. Even though conditions in the Member States still vary widely, all Europeans rightly ask themselves when and in what order the confinement measures can be lifted. Responsible planning on the ground, wisely

In [60]:
ireland_timeline = Dataset.IRELAND_ACTION_TIMELINE.fetch()

100% (410 of 410) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [63]:
ireland_timeline.reset_index(drop=True, inplace=True)

In [69]:
tt = ireland_timeline['content'].apply(lambda x:  x.replace('\n','') if type(x) == str else x)

In [71]:
tt

0                                                    ...
1                                                    ...
2                                                    ...
3                                                    ...
4                                                    ...
5                                                    ...
6                                                    ...
7                                                    ...
8                                                    ...
9                                                    ...
10                                                   ...
11                                                   ...
12                                                   ...
13                                                   ...
14                                                   ...
15                                                   ...
16                                                   ...
17                             

In [40]:
!pip install pysummarization

     |████████████████████████████████| 46 kB 4.3 MB/s eta 0:00:011
  Created wheel for pysummarization: filename=pysummarization-1.1.7-py3-none-any.whl size=45593 sha256=a78a84a269f8731d1903c2fa2cdd23ac11ea4008aa6f2ac141a2960b5906a0f9
  Stored in directory: /home/jovyan/.cache/pip/wheels/53/05/ff/48932be9a1f4292315478cb47bb6145230a2daf84d87957d8f
Successfully built pysummarization
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [41]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor

In [42]:
# Object of automatic summarization.
auto_abstractor = AutoAbstractor()
# Set tokenizer.
auto_abstractor.tokenizable_doc = SimpleTokenizer()
# Set delimiter for making a list of sentence.
auto_abstractor.delimiter_list = [".", "\n"]
# Object of abstracting and filtering document.
abstractable_doc = TopNRankAbstractor()
# Summarize document.
result_dict = auto_abstractor.summarize(document, abstractable_doc)

# Output result.
for sentence in result_dict["summarize_result"]:
    print(sentence)


 b e e n   l a u n c h e d   i n   2 0 2 1   ( R I S K - H U N T 3 R ,   P R E C I S I O N T O X ,   O N T O X –   € 6 0   m i l l i o n   f u n d i n g ,   

 7 5 0 1 0 4   R E S O U R C E   G r o u n d w a t e r   q u a l i t y   

 7 8 5 9 9 4   A I R - N B   P r e - n a t a l   e x p o s u r e   t o   

 8 1 4 0 3 0   H E A R T L A N D   H e a l t h ,   E n v i r o n m e n t ,   

 8 4 2 1 0 7   S H I N E   T h e   i m p a c t   o f   t h e   s o c i o - 

 8 4 6 7 9 4   M E T A L S   I n v e s t i g a t i n g   t h e   

 a t / q 0 0 2 p l / E R C _ C o G _ 2 0 1 4 _ M o b i l i S e n s e _ B 1 .

 8 5 7 3 4 0   U R B A N _ X   E x t e r n a l   a n d   i n t e r n a l   

 7 7 1 0 5 7   W E L L - B E I N G   T h e   d y n a m i c s   

 7 9 9 2 8 1   P R E S T O   P i l o t   s c a l e   h y b r i d   



In [32]:
def my_text_to_vector(text: str, word2vec):
    dim = len(word2vec[next(iter(word2vec))])
    return np.mean([np.zeros(dim)] + [word2vec[word] for word in text.split() if word in word2vec], axis=0)

In [34]:
text_df = pd.DataFrame(df[EU_CELLAR_TEXT_COLUMNS])
text_df.replace(np.nan, '', regex=True, inplace=True)
text_df['text'] = text_df.agg(' '.join, axis=1)
text_df.reset_index(drop=True, inplace=True)

In [32]:
text_df['text'].head(5)

NameError: name 'text_df' is not defined

In [38]:
text_df['text'].str.len().sum()

2156889442

In [43]:
text_df['text'].apply(lambda x: ' '.join(re.split('\W+', x))).str.len().sum()

1545996497

In [ ]:
text_df["emb"] = text_df["text"].apply(lambda x: my_text_to_vector(x, l2v_dict))

In [8]:
text_df[text_df['emb'].apply(lambda x: type(x) == np.float64)]

,title,content,text,emb
15,,,,NaN
41,,,,NaN
64,,,,NaN
65,,,,NaN
75,,,,NaN
...,...,...,...,...
9742,,,,NaN
9746,,,,NaN
9751,,,,NaN
9767,,,,NaN


In [12]:
matrix_df = pd.DataFrame(list(text_df['emb'].values))

TypeError: 'numpy.float64' object is not iterable

In [7]:
matrix_df

NameError: name 'matrix_df' is not defined

## Enrich dataset with new columns

In [ ]:
for new_column in EU_CELLAR_NEW_COLUMNS:
    model = get_best_model_from_ml_flow(experiment_ids=["1"], class_name=new_column)
    text_df[new_column] = model.predict(list(text_df["emb"]))


## Show result

In [ ]:
text_df[['title'] + EU_TIMELINE_NEW_COLUMNS[:]]

## Analyse results

In [ ]:
for new_column in EU_CELLAR_NEW_COLUMNS:
    n_poz = len(text_df[text_df[new_column] == 1])
    n_total = len(text_df)
    n_neg = n_total - n_poz
    print(f"{new_column}: n_neg = {round(n_neg / n_total, 2)}, n_poz = {round(n_poz / n_total, 2)}")

